SPARSE RETRIEVAL

In [1]:
!pip install -U beir

     |████████████████████████████████| 49 kB 3.2 MB/s 
     |████████████████████████████████| 4.4 MB 11.0 MB/s 
     |████████████████████████████████| 8.4 MB 15.2 MB/s 
     |████████████████████████████████| 78 kB 5.1 MB/s 
     |████████████████████████████████| 378 kB 53.7 MB/s 
     |████████████████████████████████| 2.9 MB 65.0 MB/s 
     |████████████████████████████████| 3.3 MB 57.5 MB/s 
     |████████████████████████████████| 1.2 MB 63.1 MB/s 
     |████████████████████████████████| 56 kB 5.7 MB/s 
     |████████████████████████████████| 596 kB 56.3 MB/s 
     |████████████████████████████████| 895 kB 60.8 MB/s 
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=265117 sha256=07b018775ec71b7a76ae93d72a03b710fc25857961314162b46582bc26068eeb
  Stored in directory: /root/.cache/pip/wheels/42/96/77/0829b8b2606f90f61ba10a51277629d2b615604e122ee932f4
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-an

In [2]:
import pathlib, os
import time
import pandas as pd
from tqdm.notebook import tqdm
from tqdm.autonotebook import trange
from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.sparse import SparseSearch

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """


In [3]:
def load(dataset):
  hostname = 'localhost'
  index_name = dataset
  url = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip'.format(dataset)
  out_dir = os.path.join(os.getcwd(), 'datasets')
  data_path = util.download_and_unzip(url, out_dir)
  return GenericDataLoader(data_path).load(split='test') # 'test', 'train', 'dev'

In [4]:
def retrieve(corpus, queries, qrels, model_name, batch_size):
  sparse_model = SparseSearch(models.SPARTA(model_name), batch_size=batch_size)
  retriever_sp = EvaluateRetrieval(sparse_model)
  results_sp = retriever_sp.retrieve(corpus, queries)
  return retriever_sp.evaluate(qrels, results_sp, retriever_sp.k_values)

In [5]:
def evaluate(corpus, queries, qrels, model_name, batch_size):
  ndcg, _map, recall, precision = retrieve(corpus, queries, qrels, model_name, batch_size)
  c_map = 'MAP@10'
  c_map_h = 'MAP@100'
  c_map_t = 'MAP@1000'
  c_ndcg = 'NDCG@10'
  c_ndcg_h = 'NDCG@100'
  c_ndcg_t = 'NDCG@1000'
  c_pre = 'P@10'
  c_pre_h = 'P@100'
  c_pre_t = 'P@1000'
  c_rec = 'Recall@10'
  c_rec_h = 'Recall@100'
  c_rec_t = 'Recall@1000'
  eval_dict = {
      c_map: [_map[c_map]], 
      c_map_h: [_map[c_map_h]],
      c_map_t: [_map[c_map_t]],
      c_ndcg: [ndcg[c_ndcg]], 
      c_ndcg_h: [ndcg[c_ndcg_h]],
      c_ndcg_t: [ndcg[c_ndcg_t]],
      c_pre: [precision[c_pre]], 
      c_pre_h: [precision[c_pre_h]],
      c_pre_t: [precision[c_pre_t]],
      c_rec: [recall[c_rec]],
      c_rec_h: [recall[c_rec_h]],
      c_rec_t: [recall[c_rec_t]]}
  eval_df = pd.DataFrame(data=eval_dict)
  eval_df.index = [model_name]
  return eval_df

In [6]:
model_db = 'BeIR/sparta-msmarco-distilbert-base-v1'
sf_corpus, sf_queries, sf_qrels = load('scifact')
sd_corpus, sd_queries, sd_qrels = load('scidocs')
nf_corpus, nf_queries, nf_qrels = load('nfcorpus')

/content/datasets/scifact.zip:   0%|          | 0.00/2.69M [00:00<?, ?iB/s]

  0%|          | 0/5183 [00:00<?, ?it/s]

/content/datasets/scidocs.zip:   0%|          | 0.00/136M [00:00<?, ?iB/s]

  0%|          | 0/25657 [00:00<?, ?it/s]

/content/datasets/nfcorpus.zip:   0%|          | 0.00/2.34M [00:00<?, ?iB/s]

  0%|          | 0/3633 [00:00<?, ?it/s]

**SCIFACT** - [Homepage](https://allenai.org/data/scifact) - [Paper](https://arxiv.org/abs/2004.14974)

In [7]:
evaluate(sf_corpus, sf_queries, sf_qrels, model_db, 128)

Downloading:   0%|          | 0.00/258 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/517 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

docs:   0%|          | 0/41 [00:00<?, ?it/s]

query:   0%|          | 0/3 [00:00<?, ?it/s]

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
BeIR/sparta-msmarco-distilbert-base-v1,0.66667,0.66667,0.66667,0.66667,0.66667,0.66667,0.06667,0.00667,0.00067,0.66667,0.66667,0.66667


**SCIDOCS** - [Homepage](https://allenai.org/data/scidocs) - [Paper](https://arxiv.org/abs/2004.07180)

In [8]:
evaluate(sd_corpus, sd_queries, sd_qrels, model_db, 128)

docs:   0%|          | 0/201 [00:00<?, ?it/s]

query:   0%|          | 0/8 [00:00<?, ?it/s]

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
BeIR/sparta-msmarco-distilbert-base-v1,0.035,0.05235,0.05456,0.0588,0.16545,0.21179,0.025,0.01875,0.003,0.05,0.375,0.6


**NFCORPUS** - [Homepage](https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/) - [Paper](https://www.cl.uni-heidelberg.de/~riezler/publications/papers/ECIR2016.pdf)

In [9]:
evaluate(nf_corpus, nf_queries, nf_qrels, model_db, 128)

docs:   0%|          | 0/29 [00:00<?, ?it/s]

query:   0%|          | 0/3 [00:00<?, ?it/s]

,MAP@10,MAP@100,MAP@1000,NDCG@10,NDCG@100,NDCG@1000,P@10,P@100,P@1000,Recall@10,Recall@100,Recall@1000
BeIR/sparta-msmarco-distilbert-base-v1,0.09456,0.15421,0.16379,0.35311,0.36289,0.45602,0.36667,0.08,0.01333,0.20076,0.38636,0.78157


end of fun.